In [ ]:
# Importing libraries
!pip install git+https://github.com/tensorflow/docs
from absl import logging
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import random
import re
import os
import tempfile
import ssl
import cv2
import numpy as np
import imageio
from IPython import display
from urllib import request
import re
import pprint
import glob
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-te1_z1eq
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/docs /tmp/pip-req-build-te1_z1eq
  Resolved https://github.com/tensorflow/docs to commit f743bbc0645dbdec6a92dbe7f3768a60f109e87d
  Preparing metadata (setup.py) ... done
  Created wheel for tensorflow-docs: filename=tensorflow_docs-0.0.0.dev0-py3-none-any.whl size=184468 sha256=1740fce9a227008f61143aacab468ee4e64c09330d1bc3ec3576ca47f97e62dc
  Stored in directory: /tmp/pip-ephem-wheel-cache-s5atl3lm/wheels/3b/ee/a2/ab4d36a9a4af495bcb936f3e849d4b497b65fa40548a68d6c3
Successfully built tensorflow-docs


In [ ]:
# Mounting google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Get the kinetics-600 action labels from the GitHub repository.

with open('/content/drive/MyDrive/Scomp/Visual/Dataset/labels.txt') as f:
    l = f.readlines()
    labels=len(l)

with open('/content/drive/MyDrive/Scomp/Visual/Dataset/labels.txt','r') as f_in:
    lines = f_in.read()
    list=re.split('; |, |\*|\n',lines)

In [ ]:
def crop_center_square(frame):
  y, x = frame.shape[0:2]
  min_dim = min(y, x)
  start_x = (x // 2) - (min_dim // 2)
  start_y = (y // 2) - (min_dim // 2)
  return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]

def load_video(path, max_frames=0, resize=(224, 224)):
  cap = cv2.VideoCapture(path)
  frames = []
  try:
    while True:
      ret, frame = cap.read()
      if not ret:
        break
      frame = crop_center_square(frame)
      frame = cv2.resize(frame, resize)
      frame = frame[:, :, [2, 1, 0]]
      frames.append(frame)

      if len(frames) == max_frames:
        break
  finally:
    cap.release()
  return np.array(frames) / 255.0
def to_gif(images):
  converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
  imageio.mimsave('./animation.gif', converted_images, fps=25)
  return embed.embed_file('./animation.gif')

In [ ]:
# HAPPY

path = "/content/drive/MyDrive/Scomp/Visual/Dataset/Youtube Data/Happy"
wav_files = glob.glob(path + "/*.mp4")
for i in wav_files:
    video_path=i
    sample_video = load_video(video_path)[:100]
    sample_video.shape

    hub_url = "https://tfhub.dev/tensorflow/movinet/a2/base/kinetics-600/classification/3"

    encoder = hub.KerasLayer(hub_url, trainable=True)

    inputs = tf.keras.layers.Input(
        shape=[None,None,None,3],
        dtype=tf.float32,
        name='image')

    outputs = encoder(dict(image=inputs))

    model = tf.keras.Model(inputs, outputs, name='movinet')

    def predict(sample_video):
      # Add a batch axis to the sample video.
      model_input = tf.constant(sample_video, dtype=tf.float32)[tf.newaxis, ...]

      logits= model.predict(model_input)
      logits=logits[0]
      probabilities = tf.nn.softmax(logits)

      print("Top 2 actions:")
      for i in np.argsort(probabilities)[::-1][:2]:
        print(f"  {list[i]:22}: {probabilities[i] * 100:2.2f}%")  
    result=predict(sample_video) 

1/1 [==============================] - 230s 230s/step
Top 2 actions:
  staring               : 24.57%
  crossing eyes         : 15.81%
1/1 [==============================] - 133s 133s/step
Top 2 actions:
  headbanging           : 11.75%
  hugging (not baby)    : 9.23%
1/1 [==============================] - 211s 211s/step
Top 2 actions:
  staring               : 36.18%
  photobombing          : 13.38%
1/1 [==============================] - 212s 212s/step
Top 2 actions:
  staring               : 29.08%
  crossing eyes         : 8.45%


1/1 [==============================] - 228s 228s/step
Top 2 actions:
  news anchoring        : 15.30%
  staring               : 13.15%


1/1 [==============================] - 219s 219s/step
Top 2 actions:
  crossing eyes         : 17.21%
  winking               : 11.08%
1/1 [==============================] - 221s 221s/step
Top 2 actions:
  staring               : 30.96%
  crossing eyes         : 16.77%
1/1 [==============================] - 219s 219s/step
Top 2 actions:
  massaging person's head: 66.89%
  raising eyebrows      : 3.78%
1/1 [==============================] - 219s 219s/step
Top 2 actions:
  staring               : 17.42%
  laughing              : 12.67%
1/1 [==============================] - 212s 212s/step
Top 2 actions:
  staring               : 49.64%
  crossing eyes         : 27.80%


In [ ]:
# SAD

path = "/content/drive/MyDrive/Scomp/Visual/Dataset/Youtube Data/Sad"
wav_files = glob.glob(path + "/*.mp4")

for i in wav_files:
    video_path=i
    sample_video = load_video(video_path)[:100]
    sample_video.shape

    hub_url = "https://tfhub.dev/tensorflow/movinet/a2/base/kinetics-600/classification/3"

    encoder = hub.KerasLayer(hub_url, trainable=True)

    inputs = tf.keras.layers.Input(
        shape=[None,None,None,3],
        dtype=tf.float32,
        name='image')

    outputs = encoder(dict(image=inputs))

    model = tf.keras.Model(inputs, outputs, name='movinet')

    def predict(sample_video):
      # Add a batch axis to the sample video.
      model_input = tf.constant(sample_video, dtype=tf.float32)[tf.newaxis, ...]

      logits= model.predict(model_input)
      logits=logits[0]
      probabilities = tf.nn.softmax(logits)

      print("Top 2 actions:")
      for i in np.argsort(probabilities)[::-1][:2]:
        print(f"  {list[i]:22}: {probabilities[i] * 100:2.2f}%")
  
    result=predict(sample_video) 

1/1 [==============================] - 221s 221s/step
Top 2 actions:
  crying                : 26.69%
  kissing               : 12.33%
1/1 [==============================] - 216s 216s/step
Top 2 actions:
  kissing               : 13.80%
  crying                : 8.65%
1/1 [==============================] - 219s 219s/step
Top 2 actions:
  opening door          : 9.15%
  pull ups              : 5.59%
1/1 [==============================] - 215s 215s/step
Top 2 actions:
  acting in play        : 56.07%
  singing               : 8.23%
1/1 [==============================] - 215s 215s/step
Top 2 actions:
  crying                : 10.61%
  sneezing              : 10.43%
1/1 [==============================] - 216s 216s/step
Top 2 actions:
  answering questions   : 15.65%
  using inhaler         : 14.75%
1/1 [==============================] - 215s 215s/step
Top 2 actions:
  putting on sari       : 23.39%
  celebrating           : 11.41%
1/1 [==============================] - 218s 218s/step
Top 2

In [ ]:
# SURPRISE

path = "/content/drive/MyDrive/Scomp/Visual/Dataset/Youtube Data/Surprise"
wav_files = glob.glob(path + "/*.mp4")

for i in wav_files:
    video_path=i
    sample_video = load_video(video_path)[:100]
    sample_video.shape

    hub_url = "https://tfhub.dev/tensorflow/movinet/a2/base/kinetics-600/classification/3"

    encoder = hub.KerasLayer(hub_url, trainable=True)

    inputs = tf.keras.layers.Input(
        shape=[None,None,None,3],
        dtype=tf.float32,
        name='image')

    outputs = encoder(dict(image=inputs))

    model = tf.keras.Model(inputs, outputs, name='movinet')

    def predict(sample_video):
      # Add a batch axis to the sample video.
      model_input = tf.constant(sample_video, dtype=tf.float32)[tf.newaxis, ...]

      logits= model.predict(model_input)
      logits=logits[0]
      probabilities = tf.nn.softmax(logits)

      print("Top 2 actions:")
      for i in np.argsort(probabilities)[::-1][:2]:
        print(f"  {list[i]:22}: {probabilities[i] * 100:2.2f}%")
    
    predict(sample_video) 

1/1 [==============================] - 215s 215s/step
Top 2 actions:
  sleeping              : 16.07%
  slapping              : 14.12%
1/1 [==============================] - 218s 218s/step
Top 2 actions:
  wrapping present      : 26.13%
  opening present       : 10.17%
1/1 [==============================] - 219s 219s/step
Top 2 actions:
  moving furniture      : 21.38%
  making the bed        : 12.83%
1/1 [==============================] - 213s 213s/step
Top 2 actions:
  sneezing              : 38.85%
  burping               : 28.97%
1/1 [==============================] - 221s 221s/step
Top 2 actions:
  photobombing          : 11.85%
  singing               : 8.49%
1/1 [==============================] - 217s 217s/step
Top 2 actions:
  opening door          : 88.57%
  opening refrigerator  : 2.08%
1/1 [==============================] - 222s 222s/step
Top 2 actions:
  sneezing              : 9.95%
  crying                : 8.85%
1/1 [==============================] - 218s 218s/step
Top 2

In [ ]:
# ANGER

path = "/content/drive/MyDrive/Scomp/Visual/Dataset/Youtube Data/Anger"
wav_files = glob.glob(path + "/*.mp4")

for i in wav_files:
    video_path=i
    sample_video = load_video(video_path)[:100]
    sample_video.shape

    hub_url = "https://tfhub.dev/tensorflow/movinet/a2/base/kinetics-600/classification/3"

    encoder = hub.KerasLayer(hub_url, trainable=True)

    inputs = tf.keras.layers.Input(
        shape=[None,None,None,3],
        dtype=tf.float32,
        name='image')

    outputs = encoder(dict(image=inputs))

    model = tf.keras.Model(inputs, outputs, name='movinet')

    def predict(sample_video):
      # Add a batch axis to the sample video.
      model_input = tf.constant(sample_video, dtype=tf.float32)[tf.newaxis, ...]

      logits= model.predict(model_input)
      logits=logits[0]
      probabilities = tf.nn.softmax(logits)

      print("Top 2 actions:")
      for i in np.argsort(probabilities)[::-1][:2]:
        print(f"  {list[i]:22}: {probabilities[i] * 100:2.2f}%")
   
    predict(sample_video) 

1/1 [==============================] - 224s 224s/step
Top 2 actions:
  yawning               : 50.46%
  sneezing              : 15.21%
1/1 [==============================] - 219s 219s/step
Top 2 actions:
  sneezing              : 15.53%
  cracking neck         : 8.06%
1/1 [==============================] - 214s 214s/step
Top 2 actions:
  playing paintball     : 10.38%
  catching fish         : 3.80%
1/1 [==============================] - 219s 219s/step
Top 2 actions:
  smoking               : 12.70%
  sleeping              : 6.73%


1/1 [==============================] - 215s 215s/step
Top 2 actions:
  crying                : 18.84%
  burping               : 8.66%


1/1 [==============================] - 226s 226s/step
Top 2 actions:
  motorcycling          : 45.77%
  riding scooter        : 8.88%
1/1 [==============================] - 212s 212s/step
Top 2 actions:
  staring               : 22.07%
  whistling             : 12.23%
1/1 [==============================] - 208s 208s/step
Top 2 actions:
  smoking               : 31.38%
  testifying            : 16.76%
1/1 [==============================] - 215s 215s/step
Top 2 actions:
  staring               : 47.72%
  crying                : 13.78%
1/1 [==============================] - 207s 207s/step
Top 2 actions:
  trimming or shaving beard: 51.96%
  tasting beer          : 13.31%


In [ ]:
# NEUTRAL

path = "/content/drive/MyDrive/Scomp/Visual/Dataset/Youtube Data/Neutral"
wav_files = glob.glob(path + "/*.mp4")

for i in wav_files:
    video_path=i
    sample_video = load_video(video_path)[:100]
    sample_video.shape

    hub_url = "https://tfhub.dev/tensorflow/movinet/a2/base/kinetics-600/classification/3"

    encoder = hub.KerasLayer(hub_url, trainable=True)

    inputs = tf.keras.layers.Input(
        shape=[None,None,None,3],
        dtype=tf.float32,
        name='image')

    outputs = encoder(dict(image=inputs))

    model = tf.keras.Model(inputs, outputs, name='movinet')

    def predict(sample_video):
      # Add a batch axis to the sample video.
      model_input = tf.constant(sample_video, dtype=tf.float32)[tf.newaxis, ...]

      logits= model.predict(model_input)
      logits=logits[0]
      probabilities = tf.nn.softmax(logits)

      print("Top 2 actions:")
      for i in np.argsort(probabilities)[::-1][:2]:
        print(f"  {list[i]:22}: {probabilities[i] * 100:2.2f}%")
    
    predict(sample_video) 

1/1 [==============================] - 213s 213s/step
Top 2 actions:
  news anchoring        : 56.27%
  answering questions   : 6.49%
1/1 [==============================] - 211s 211s/step
Top 2 actions:
  staring               : 19.67%
  finger snapping       : 13.37%
1/1 [==============================] - 208s 208s/step
Top 2 actions:
  massaging neck        : 17.56%
  staring               : 14.10%
1/1 [==============================] - 203s 203s/step
Top 2 actions:
  staring               : 19.39%
  singing               : 13.24%


1/1 [==============================] - 127s 127s/step
Top 2 actions:
  testifying            : 28.37%
  news anchoring        : 19.24%


1/1 [==============================] - 203s 203s/step
Top 2 actions:
  news anchoring        : 49.74%
  presenting weather forecast: 33.60%
1/1 [==============================] - 202s 202s/step
Top 2 actions:
  waving hand           : 16.05%
  slapping              : 15.16%
1/1 [==============================] - 228s 228s/step
Top 2 actions:
  testifying            : 61.02%
  answering questions   : 16.07%
1/1 [==============================] - 208s 208s/step
Top 2 actions:
  news anchoring        : 88.49%
  presenting weather forecast: 5.36%
1/1 [==============================] - 215s 215s/step
Top 2 actions:
  news anchoring        : 14.35%
  testifying            : 11.03%
